<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=12e5989cdea1cf36c7facf4730a47a90f4e7c284671da03c8c1e0072cf845a8e
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-19 11:00:27
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  6.17 L
Current:  1.43 C
-------------------
Today PnL: -10.52 K (-0.07%)
Current PnL: -16.86 L (-11.34%)
CY Booked + Current PnL: -5.42 L (-3.64%)
-------------------
Total profit:  2.04 L
Total loss:  -18.90 L
-------------------
Total Booked + Current PnL: 21.56 L (17.72%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.01%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.24 C
Est FTT PnL: 81.37 L (57.01%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.87%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.00,62.0,H-MC,3.47,195090.0,30420.0,12915.0,-0.52,18.47,6.62,26.31,89.0,2.36,1.43,27.93,X40N,NTT,AC
76,TTKPRESTIG,770.00,102.13,51.0,M-SC,3.71,87115.0,-13662.0,13755.0,-1.28,-13.56,15.79,0.09,245.0,-0.99,0.64,14.45,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.72,60.0,H-MC,2.69,213555.0,21813.0,15953.0,-0.18,11.38,7.47,19.69,99.0,1.37,1.56,18.95,XY25,NTT,AC
49,NATIONALUM,244.55,-43.66,65.0,H-MC,7.95,112095.0,11591.0,17520.0,0.09,11.53,15.63,28.96,79.0,0.66,0.82,49.42,MH,ATH,METALS
56,RELIANCE,1533.00,-12.54,55.0,H-LC,4.95,219196.0,8830.0,19947.0,-0.70,4.20,9.10,13.68,37.0,0.44,1.60,21.41,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-175.00,71.0,M-SC,3.34,86436.0,-15774.0,109160.0,3.50,-15.43,126.29,91.37,208.0,-0.14,0.63,40.31,XY25,NTT,FINANCE
59,SAIL,228.0,48.36,70.0,M-MC,7.55,233693.0,8731.0,158935.0,1.90,3.88,68.01,74.53,192.0,0.05,1.71,37.39,XY24,BTT,STEEL
6,ASIANTILES,137.0,7100.00,68.0,L-SC,5.23,78994.0,-14816.0,91436.0,1.84,-15.79,115.75,81.67,269.0,-0.16,0.58,52.98,XR,NTT,CERAMICS
63,SHALBY,327.0,1120.10,71.0,M-SC,4.65,165870.0,-15273.0,60742.0,1.71,-8.43,36.62,25.10,235.0,-0.25,1.21,32.93,XY24,NTT,HEALTHCARE
78,UNIONBANK,163.0,-13.18,65.0,M-LC,7.04,157816.0,16976.0,27523.0,1.62,12.05,17.44,31.59,66.0,0.62,1.16,41.73,XY24,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,INDIAMART,4850.92,-52.96,35.0,H-SC,11.49,131382.0,8046.0,125719.0,-1.75,6.52,95.69,108.45,119.0,0.06,0.96,30.33,AR,ATH,MISC
53,RAJESHEXPO,518.00,1857.07,61.0,L-SC,2.66,53400.0,-83777.0,83870.0,-1.68,-61.07,157.06,0.07,267.0,-1.00,0.39,32.81,OX40N,NTT,JEWELLERY
80,VAIBHAVGBL,521.00,63.09,67.0,H-SC,6.62,141378.0,-41397.0,152462.0,-1.59,-22.65,107.84,60.77,125.0,-0.27,1.04,26.63,XR,NTT,JEWELLERY
55,RELAXO,1176.00,-32.71,53.0,H-SC,1.93,91744.0,-53416.0,129341.0,-1.49,-36.80,140.98,52.31,136.0,-0.41,0.67,22.80,X40N,NTT,FOOTWEAR
52,QUESS,986.00,-40.50,47.0,M-SC,39.83,58878.0,-6128.0,157057.0,-1.38,-9.43,266.75,232.18,198.0,-0.04,0.43,2.54,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,ICICIPRULI,790.00,-20.42,47.0,H-MC,2.17,136922.0,1134.0,41624.0,0.14,0.84,30.40,31.48,107.0,0.03,1.00,13.05,X40,ATH,INSURANCE
29,HAVELLS,2069.16,0.77,60.0,H-MC,1.93,249740.0,2009.0,73049.0,0.41,0.81,29.25,30.30,92.0,0.03,1.83,14.21,X40,ATH,ELECTRICAL
51,PGHH,17973.08,-30.81,52.0,H-MC,4.09,200130.0,-690.0,69465.0,-0.08,-0.34,34.71,34.25,80.0,-0.01,1.47,4.84,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,102.13,51.0,M-SC,3.71,87115.0,-13662.0,13755.0,-1.28,-13.56,15.79,0.09,245.0,-0.99,0.64,14.45,OX40N,NTT,DURABLES
77,UJJIVANSFB,60.0,94.87,53.0,H-SC,9.41,122958.0,-19521.0,39039.0,0.07,-13.70,31.75,13.70,163.0,-0.50,0.90,45.12,OX40N,NTT,BANKS
44,KANSAINER,340.0,-66.69,66.0,H-SC,1.24,229284.0,-40383.0,76718.0,0.60,-14.98,33.46,13.47,138.0,-0.53,1.68,15.97,XY24,NTT,PAINTS
18,CERA,9475.0,-20.05,46.0,H-SC,2.10,145786.0,-30117.0,72135.0,-0.14,-17.12,49.48,23.89,149.0,-0.42,1.07,25.91,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-19.85,52.0,H-MC,7.46,108328.0,-23507.0,65733.0,-0.56,-17.83,60.68,32.03,98.0,-0.36,0.79,21.34,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,48.36,70.0,M-MC,7.55,233693.0,8731.0,158935.0,1.90,3.88,68.01,74.53,192.0,0.05,1.71,37.39,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.00,62.0,H-MC,3.47,195090.0,30420.0,12915.0,-0.52,18.47,6.62,26.31,89.0,2.36,1.43,27.93,X40N,NTT,AC
83,VOLTAS,1530.00,1.72,60.0,H-MC,2.69,213555.0,21813.0,15953.0,-0.18,11.38,7.47,19.69,99.0,1.37,1.56,18.95,XY25,NTT,AC
17,CAMS,5211.76,1.71,69.0,H-SC,3.44,115556.0,13552.0,30368.0,1.27,13.29,26.28,43.06,122.0,0.45,0.85,32.68,X40N,ATH,MISC
29,HAVELLS,2069.16,0.77,60.0,H-MC,1.93,249740.0,2009.0,73049.0,0.41,0.81,29.25,30.30,92.0,0.03,1.83,14.21,X40,ATH,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,65.0,H-MC,7.95,112095.0,11591.0,17520.0,0.09,11.53,15.63,28.96,79.0,0.66,0.82,49.42,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.66,51.0,M-SC,0.65,100215.0,8716.0,70030.0,0.06,9.53,69.88,86.07,223.0,0.12,0.73,45.53,XR,NTT,DURABLES
85,WIPRO,420.00,-9.71,58.0,M-LC,5.66,158633.0,7688.0,101763.0,-0.42,5.09,64.15,72.51,53.0,0.08,1.16,11.42,XR,NTT,IT
36,INDIAMART,4850.92,-52.96,35.0,H-SC,11.49,131382.0,8046.0,125719.0,-1.75,6.52,95.69,108.45,119.0,0.06,0.96,30.33,AR,ATH,MISC
11,BANKINDIA,190.00,-27.95,66.0,H-MC,6.33,185052.0,5244.0,107164.0,1.10,2.92,57.91,62.52,88.0,0.05,1.35,38.00,XR,NTT,BANKS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-16.58,41.0,H-LC,6.59,300562.0,-15280.0,127949.0,-0.52,-4.84,42.57,35.67,5.0,-0.12,2.20,8.35,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-12.67,50.0,H-LC,7.80,220590.0,-31178.0,84398.0,1.13,-12.38,38.26,21.13,27.0,-0.37,1.62,18.65,X40,ATH,PAINTS
41,ITC,452.00,-37.92,50.0,H-LC,2.38,200612.0,474.0,20422.0,-0.32,0.24,10.18,10.44,4.0,0.02,1.47,5.81,X40,NTT,FMCG
30,HCLTECH,1943.91,1.51,51.0,H-LC,9.71,232234.0,-9690.0,72968.0,-0.98,-4.01,31.42,26.15,8.0,-0.13,1.70,9.74,X40,ATH,IT
43,JIOFIN,387.00,0.39,52.0,H-LC,13.14,272706.0,5341.0,60104.0,-0.08,2.00,22.04,24.48,48.0,0.09,2.00,58.07,XY24,BTT,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,0.29,70.0,H-LC,1.48,248738.0,2587.0,76611.0,0.05,1.05,30.80,32.18,16.0,0.03,1.82,37.47,X200,ATH,IT
41,ITC,452.0,-37.92,50.0,H-LC,2.38,200612.0,474.0,20422.0,-0.32,0.24,10.18,10.44,4.0,0.02,1.47,5.81,X40,NTT,FMCG
50,NESTLEIND,1377.0,-9.24,52.0,H-LC,4.18,279396.0,13970.0,42831.0,-1.25,5.26,15.33,21.40,11.0,0.33,2.05,12.98,XY25,NTT,FMCG
56,RELIANCE,1533.0,-12.54,55.0,H-LC,4.95,219196.0,8830.0,19947.0,-0.70,4.20,9.10,13.68,37.0,0.44,1.60,21.41,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-19.78,52.0,H-LC,5.71,214200.0,9700.0,31723.0,-0.93,4.74,14.81,20.25,10.0,0.31,1.57,13.89,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNITDSPR,1644.00,-12.66,53.0,H-LC,7.75,229018.0,-5126.0,53751.0,0.31,-2.19,23.47,20.77,86.0,-0.10,1.68,4.58,X40N,NTT,BREWERIES
73,TCS,4476.75,-25.47,57.0,H-LC,12.30,297012.0,-48944.0,123795.0,-0.54,-14.15,41.68,21.64,1.0,-0.40,2.17,5.22,X40,ATH,IT
41,ITC,452.00,-37.92,50.0,H-LC,2.38,200612.0,474.0,20422.0,-0.32,0.24,10.18,10.44,4.0,0.02,1.47,5.81,X40,NTT,FMCG
82,VBL,671.64,-16.58,41.0,H-LC,6.59,300562.0,-15280.0,127949.0,-0.52,-4.84,42.57,35.67,5.0,-0.12,2.20,8.35,X40N,ATH,FMCG
30,HCLTECH,1943.91,1.51,51.0,H-LC,9.71,232234.0,-9690.0,72968.0,-0.98,-4.01,31.42,26.15,8.0,-0.13,1.70,9.74,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,7969.85,3.60,65.0,H-LC,26.99,166340.0,-19755.0,232161.0,0.27,-10.62,139.57,114.13,15.0,-0.09,1.22,22.50,AR,ATH,ELECTRICAL
41,ITC,452.00,-37.92,50.0,H-LC,2.38,200612.0,474.0,20422.0,-0.32,0.24,10.18,10.44,4.0,0.02,1.47,5.81,X40,NTT,FMCG
19,CIPLA,1795.00,-19.78,52.0,H-LC,5.71,214200.0,9700.0,31723.0,-0.93,4.74,14.81,20.25,10.0,0.31,1.57,13.89,X40N,BTT,PHARMA
56,RELIANCE,1533.00,-12.54,55.0,H-LC,4.95,219196.0,8830.0,19947.0,-0.70,4.20,9.10,13.68,37.0,0.44,1.60,21.41,XY25,NTT,REFINERIES
5,ASIANPAINT,3465.66,-12.67,50.0,H-LC,7.80,220590.0,-31178.0,84398.0,1.13,-12.38,38.26,21.13,27.0,-0.37,1.62,18.65,X40,ATH,PAINTS


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.47,57.0,H-LC,12.30,297012.0,-48944.0,123795.0,-0.54,-14.15,41.68,21.64,1.0,-0.40,2.17,5.22,X40,ATH,IT
39,INFY,2275.00,-16.73,54.0,H-LC,8.19,324569.0,11681.0,160013.0,-1.09,3.73,49.30,54.87,3.0,0.07,2.38,10.63,X40,BTT,IT
41,ITC,452.00,-37.92,50.0,H-LC,2.38,200612.0,474.0,20422.0,-0.32,0.24,10.18,10.44,4.0,0.02,1.47,5.81,X40,NTT,FMCG
82,VBL,671.64,-16.58,41.0,H-LC,6.59,300562.0,-15280.0,127949.0,-0.52,-4.84,42.57,35.67,5.0,-0.12,2.20,8.35,X40N,ATH,FMCG
1,ABB,7934.00,-37.53,68.0,H-LC,9.85,261182.0,-437.0,119647.0,0.10,-0.17,45.81,45.57,7.0,-0.00,1.91,11.14,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7100.00,68.0,L-SC,5.23,78994.0,-14816.0,91436.0,1.84,-15.79,115.75,81.67,269.0,-0.16,0.58,52.98,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-41.65,52.0,L-SC,42.27,85037.0,-28512.0,68565.0,0.75,-25.11,80.63,35.27,268.0,-0.42,0.62,108.13,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-175.00,71.0,M-SC,3.34,86436.0,-15774.0,109160.0,3.50,-15.43,126.29,91.37,208.0,-0.14,0.63,40.31,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-3.83,49.0,H-SC,12.50,89440.0,-11523.0,106058.0,0.23,-11.41,118.58,93.63,148.0,-0.11,0.65,33.22,SR,ATH,CHEMICALS
48,MASFIN,398.61,-17.34,51.0,H-SC,12.98,94185.0,-3795.0,25402.0,-0.05,-3.87,26.97,22.05,152.0,-0.15,0.69,36.90,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.0,1120.10,71.0,M-SC,4.65,165870.0,-15273.0,60742.0,1.71,-8.43,36.62,25.10,235.0,-0.25,1.21,32.93,XY24,NTT,HEALTHCARE
60,SAMMAANCAP,326.0,-175.00,71.0,M-SC,3.34,86436.0,-15774.0,109160.0,3.50,-15.43,126.29,91.37,208.0,-0.14,0.63,40.31,XY25,NTT,FINANCE
7,ATULAUTO,844.0,3851.35,61.0,M-SC,5.35,123192.0,-24176.0,75997.0,-1.16,-16.41,61.69,35.16,236.0,-0.32,0.90,25.83,XY24,NTT,AUTO
74,TITAGARH,1548.0,0.96,68.0,H-SC,3.16,165602.0,-26413.0,105290.0,-0.02,-13.76,63.58,41.08,158.0,-0.25,1.21,40.33,XY24,NTT,ENGINEERING
80,VAIBHAVGBL,521.0,63.09,67.0,H-SC,6.62,141378.0,-41397.0,152462.0,-1.59,-22.65,107.84,60.77,125.0,-0.27,1.04,26.63,XR,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.06
1,25,43.90
2,50,74.47


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.54
LC    32.12
MC    23.31
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.36
X40      15.40
X40N     11.95
XR       11.81
AR        8.74
XY25      8.26
OX40N     8.19
X200      1.82
MH        1.70
X5K       1.45
SR        1.29
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.29
H-LC    25.39
H-MC    20.29
M-SC    13.66
M-LC     5.66
M-MC     2.71
L-SC     1.59
L-LC     1.07
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.62,-12.93,74.50
FMCG,11.88,-4.46,41.01
FINANCE,9.76,-15.24,60.38
MISC,7.47,-12.64,75.27
BANKS,6.34,-15.24,75.87
PAINTS,5.89,-14.73,31.83
ELECTRICAL,5.79,-4.17,61.59
AC,3.71,4.59,11.39
INSURANCE,3.44,-6.97,43.66


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2833611.0,22
AR,1301834.0,9
XR,1297011.0,14
X40,746064.0,10
X40N,573374.0,9
OX40N,527332.0,10
XY25,336110.0,6
SR,244497.0,2
X5K,120068.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3409244.0,29
M-SC,1377896.0,17
H-LC,1226320.0,14
H-MC,1095346.0,15
M-LC,378286.0,4
M-MC,307627.0,2
L-SC,243871.0,3
L-MC,60441.0,1
L-LC,38471.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1022430.0      6
M-SC       XY24       795472.0      7
H-SC       AR         788529.0      5
           XR         753152.0      7
H-MC       XY24       491523.0      4
H-LC       X40        461596.0      5
           AR         351808.0      2
M-MC       XY24       307627.0      2
H-SC       X40N       285538.0      4
           OX40N      251628.0      4
           SR         244497.0      2
H-LC       X40N       213423.0      3
H-MC       X40        213292.0      4
M-SC       AR         161497.0      2
L-SC       XR         160001.0      2
M-LC       XY24       156455.0      2
M-SC       OX40N      126101.0      4
H-MC       XY25       125701.0      2
M-LC       X5K        120068.0      1
M-SC       XR         114490.0      2
           XY25       109160.0      1
H-MC       XR         107164.0      1
M-LC       XR         101763.0      1
L-SC       OX40N       83870.0      1
H-LC       X200        76611.0      1
H-MC       X40N        74413.0      2
M-SC       X40         71176.0      1
H-MC       OX40N       65733.0      1
H-SC       MH          63470.0      1
H-LC       XY25        62778.0      2
L-MC       XR          60441.0      1
H-LC       XY24        60104.0      1
L-LC       XY25        38471.0      1
H-MC       MH          17520.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
